In [3]:
# ✅ Install dependencies if needed
%pip install sqlalchemy psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [5]:
# 🔌 Connect to PostgreSQL
from sqlalchemy import create_engine, text

# 👉 Edit these for your environment
DB_USER = "username"    # Replace with your PostgreSQL username
DB_PASS = "passqord"    # Replace with your PostgreSQL password
DB_HOST = "localhost"   
DB_PORT = "5432" # Default PostgreSQL port
DB_NAME = "name_of_database"  # Replace with your database name

DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

In [9]:
from sqlalchemy import text

sql_path = "ecoimpact_dump.sql"

with open(sql_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

valid_sql = []
in_copy_block = False

for line in lines:
    stripped = line.strip()
    if stripped.lower().startswith("copy ") and "from stdin" in stripped.lower():
        in_copy_block = True
        continue
    if stripped == "\\.":
        in_copy_block = False
        continue
    if in_copy_block:
        continue
    if (
        not stripped
        or stripped.startswith("--")
        or any(stripped.startswith(prefix) for prefix in ("Name:", "Type:", "Schema:", "Owner:"))
    ):
        continue
    valid_sql.append(line)

sql_cleaned = "".join(valid_sql)

with engine.begin() as conn:  # <-- FIXED HERE
    for statement in sql_cleaned.split(";"):
        stmt = statement.strip()
        if stmt:
            try:
                conn.execute(text(stmt + ";"))
            except Exception as e:
                print(f"⚠️ Skipped: {e}")

print("✅ Cleaned SQL schema loaded!")


⚠️ Skipped: (psycopg2.errors.DuplicateTable) relation "Plastic_Waste_Data" already exists

[SQL: CREATE TABLE public."Plastic_Waste_Data" (
    "Country" text,
    "Total_Waste" double precision,
    "Main_Sources" text,
    "Recycling_Rate" double precision,
    "Per_Capita_Waste_KG" double precision,
    "Coastal_Waste_Risk" text,
    "Country_Code" text
);]
(Background on this error at: https://sqlalche.me/e/14/f405)
⚠️ Skipped: (psycopg2.errors.InFailedSqlTransaction) current transaction is aborted, commands ignored until end of transaction block

[SQL: ALTER TABLE public."Plastic_Waste_Data" OWNER TO postgres;]
(Background on this error at: https://sqlalche.me/e/14/2j85)
⚠️ Skipped: (psycopg2.errors.InFailedSqlTransaction) current transaction is aborted, commands ignored until end of transaction block

[SQL: CREATE TABLE public.air_super (
    country text,
    country_code text,
    year integer,
    total double precision,
    coal double precision,
    oil double precision,
   

In [ ]:
# ✅ Verify tables
import pandas as pd

tables = pd.read_sql("SELECT table_name FROM information_schema.tables WHERE table_schema='public';", engine)
tables